In [ ]:
from typing import List, Set

# Define a type alias for Sudoku
Sudoku = List[List[Set[int]]]

def create_sudoku() -> Sudoku:
    # Create an empty Sudoku grid
    sudoku = [[set(range(1, 10)) for _ in range(9)] for _ in range(9)]
    return sudoku

def display_sudoku(sudoku: Sudoku) -> None:
    # Display the Sudoku grid
    for row in sudoku:
        for cell in row:
            if len(cell) > 1:
                print(cell, end=' ')
            else:
                print(cell.pop() if cell else ' ', end=' ')
        print()


sudoku = create_sudoku()
display_sudoku(sudoku)

def propagate_constraints(sudoku: Sudoku, row: int, col: int) -> List[tuple[int, int]]:
    # Propagate constraints from a single cell
    new_cells = []
    value = sudoku[row][col].pop()  # Retrieve the known value
    sudoku[row][col] = {value}  # Update the cell with the known value

    # Remove the value from the same row and column
    for i in range(9):
        if value in sudoku[row][i] and i != col:
            sudoku[row][i].remove(value)
            if len(sudoku[row][i]) == 1:
                new_cells.append((row, i))
        if value in sudoku[i][col] and i != row:
            sudoku[i][col].remove(value)
            if len(sudoku[i][col]) == 1:
                new_cells.append((i, col))

    # Remove the value from the same 3x3 block
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(start_row, start_row + 3):
        for j in range(start_col, start_col + 3):
            if value in sudoku[i][j] and (i, j) != (row, col):
                sudoku[i][j].remove(value)
                if len(sudoku[i][j]) == 1:
                    new_cells.append((i, j))

    return new_cells


# Create a Sudoku grid with a known value
sudoku = create_sudoku()
sudoku[0][0] = {5}

# Propagate constraints from the known value
new_cells = propagate_constraints(sudoku, 0, 0)

# Display the updated Sudoku grid
display_sudoku(sudoku)

def solve_sudoku(sudoku: Sudoku) -> bool:
    # Find the next empty cell
    for row in range(9):
        for col in range(9):
            if len(sudoku[row][col]) > 1:
                # Try each possible value in the cell
                for value in sudoku[row][col].copy():
                    sudoku_copy = [row.copy() for row in sudoku]  # Make a copy of the Sudoku grid
                    sudoku_copy[row][col] = {value}  # Set the value in the current cell

                    # Propagate constraints and recursively solve the updated Sudoku
                    new_cells = propagate_constraints(sudoku_copy, row, col)
                    if not new_cells:
                        # No contradiction, continue solving recursively
                        if solve_sudoku(sudoku_copy):
                            # Solution found, update the original grid and return
                            for i, j in new_cells:
                                sudoku[i][j] = sudoku_copy[i][j]
                            return True
                return False  # No valid value found, backtrack

    return True  # All cells filled, solution found

sudoku = create_sudoku()
# Fill in some known values (0 denotes an empty cell)
sudoku[0][0] = {5}
sudoku[0][1] = {3}
sudoku[1][0] = {6}
sudoku[2][2] = {8}
# ...

# Solve the Sudoku
if solve_sudoku(sudoku):
    print("Solution found:")
    display_sudoku(sudoku)
else:
    print("No solution exists.")
